# Convert a CSV file with Location - Daily Max Discharge to Stage Height

In [17]:
import numpy as np
import pandas as pd
import os

In [18]:
###  STEPS TO CONVERT daily_max_discharge (m3/sec or m3/hr) to stage height (m)
# STEP 1
# 1. Load the daily max discharge data 
# 2. Each location is within a catchment that is included in HUC6 region.
# 3. For a location, load a hydrotable (rating curve table) - Hydrotable is unique per huc6 region
# 3. Look up for the catchment within the hydrotable
# 3. Look up discharge value in the catchment-indexed hydrotable and find corresponding stage height value
#   (Interpolate if necessary)
#
# STEP 2 (somewhat manual work)
# Remove NaN
# Remove locations with other issues

# Step 1. Get stage height from daily max discharge

In [19]:
### Input
# 1. dff: rating curve dataframe indexed by catchment id
# 2. discharge: daily_max_discharge of a location
### Output
# stage height value

def get_stage(dff, discharge): 
        
    dff = dff.reset_index()
    col_discharge = dff.columns.tolist()[len(dff.columns)-1]
    array = np.array(dff[col_discharge])
    
    # check if the first value of array is nan
    if np.isnan(array[0]):
        dff.loc[0, col_discharge] = 0
        array[0] = 0
       
    list_output = []
    idx = (np.abs(array - discharge)).argmin()
    list_output.append(idx)

    if array[idx] - discharge < 0:
        if idx == len(dff)-1:
            idx2 = idx - 1 
        else:
            idx2 = idx + 1
    else:
        idx2 = idx - 1
        
    list_output.append(idx2)
    list_output.sort()

    # interporlate
    slope = (dff.iloc[list_output[1]].Stage - dff.iloc[list_output[0]].Stage) / (dff.iloc[list_output[1]][col_discharge] - dff.iloc[list_output[0]][col_discharge])
    stage = slope * (discharge - dff.iloc[list_output[0]][col_discharge]) + dff.iloc[list_output[0]].Stage

    return stage

In [20]:
### Load max discharge csv file
path = '/home/jupyter/tacc-work/nwm_output/'
file = 'location_maxdischarge.csv'
df_discharge = pd.read_csv(path + file, index_col = 0)
format_col = df_discharge.columns[:9].tolist()
scenario_col = df_discharge.columns[9:].tolist()
df_stage = df_discharge[format_col]

# Find stage height of the locations within df_stage
print(df_stage.shape)
df_stage.head()

(897, 9)


,ID,NID,LATITUDE,LONGITUDE,TYPE,HUC6,HAND,DEM,CATCHID
0,7477418,h1,29.953158,-96.260098,HOSPITAL,120701,21.731787,88.494926,3122636
1,73678962,h2,29.694502,-96.788134,HOSPITAL,121001,17.844019,118.192200,7845289
2,16977434,h3,29.587759,-96.345285,HOSPITAL,120903,6.686262,52.635021,3763126
3,11578934,h4,29.694518,-96.543311,HOSPITAL,120903,7.384333,60.080364,3762970
4,44377469,h5,29.576329,-95.770937,HOSPITAL,120701,3.750920,29.634687,3123672


In [21]:
print(scenario_col)

['25-01', '25-02', '25-03', '25-04', '25-05', '25-06', '25-07', '25-08', '25-09', '25-10', '25-11', '25-12', '25-13', '25-14', '25-15', '25-16', '25-17', '25-18', '25-19', '25-20', '25-21', '25-22', '25-23', '25-24', '25-25']


In [22]:
this_huc6 = '120100'
url = '/home/jupyter/tacc-work/hand/version-0.2.1/files/'
set_url = url + str(this_huc6) + '/'
table_name = ('hydrogeo-fulltable-%s.csv' % this_huc6 )
df_ratingcurve = pd.read_csv(set_url + table_name)

In [23]:
df_ratingcurve.shape

(677778, 15)

In [24]:
col_counter = 0
error_catch = []
error_location = []
for col in scenario_col:
    col_counter = col_counter + 1
    print('Running column: ', col, '(' , col_counter, '/', len(scenario_col), ')')
    this_scenario = col
    format_col = df_discharge.columns[:9].tolist()
    format_col.append(this_scenario)
    this_df_discharge = df_discharge[format_col]

    huc6_list = this_df_discharge.HUC6.unique().tolist()

    dict_stage = {} # NID: stage
    num_locations = 0
    for this_huc6 in huc6_list:
        print('   huc6: ', this_huc6)
        url = '/home/jupyter/tacc-work/hand/version-0.2.1/files/'
        set_url = url + str(this_huc6) + '/'
        table_name = ('hydrogeo-fulltable-%s.csv' % this_huc6 )
        df_ratingcurve = pd.read_csv(set_url + table_name)
        this_df = this_df_discharge[this_df_discharge.HUC6 == this_huc6] # Dataframe includes all locations within "this_huc6"
        num_locations = num_locations + len(this_df)
    
        # Find catchment and stage height for each location within this_df
        for i in range(len(this_df)):
            this_location = this_df.iloc[i].NID
            this_catchid = this_df.iloc[i].CATCHID
            this_discharge = this_df.iloc[i][col]
            dff = df_ratingcurve[df_ratingcurve.CatchId == this_catchid]
            
            if sum(dff['Discharge (m3s-1)']) == 0:
                error_catch.append(this_catchid)
                error_location.append(this_location)
                print(this_location, this_catchid)
                continue
                
            dict_stage[this_location] = get_stage(dff, this_discharge)
        
    print(col, num_locations)

    df_stage[col] = df_stage['NID'].map(dict_stage)

output_file_name = 'stage_output.csv'
df_stage.to_csv(path + output_file_name)

Running column:  25-01 ( 1 / 25 )
   huc6:  120701
   huc6:  121001
   huc6:  120903


KeyboardInterrupt: 

# Step 2. Clean the stage output dataframe 

## Find NaN rows

In [9]:
## Location with discharge values = 0 
# This is the locations with rating curve slope 0
# Not NWM region
missing_locations = [['h142', 1524235],['h144', 1569042],['h146', 1569042],['n249', 1563632],['n597', 1569056],['n601', 1564542],['n603', 1569054],['n604', 1564542],['n607', 1569042],['n608', 1569036],['n610', 1569042],['n612', 1569054],['n615', 1569042]]
len(missing_locations)

In [11]:
missing_data = []
for pair in missing_locations:
    lat = df_stage.loc[df_stage.NID == pair[0]].LATITUDE.values[0]
    lon = df_stage.loc[df_stage.NID == pair[0]].LONGITUDE.values[0]
    pair.append(lat)
    pair.append(lon)
    missing_data.append(pair)
cols = ['NID', 'CATCHID', 'LATITUDE', 'LONGITUDE']
df_missing = pd.DataFrame(missing_data, columns=cols)
df_missing.to_csv(path + 'outofnwm.csv')

In [49]:
## Find all NaN locations
is_NaN = df_stage.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_stage[row_has_NaN]
rows_with_NaN.shape

(33, 34)

In [17]:
rows_with_NaN.to_csv(path + 'na.csv')

In [44]:
## Locations with discharge value = ******
# out of 33 NaN locations, 13 are out of NWM region
# what are the remaining 20?
location_na = rows_with_NaN.NID
location_slope = df_missing.NID
missing_unknown = []
for i in location_na:
    if np.isin(i, location_slope):
        continue
    else:
        missing_unknown.append(i)
print(missing_unknown)

['h23', 'h34', 'h40', 'h42', 'h82', 'h93', 'n127', 'n170', 'n201', 'n285', 'n319', 'n353', 'n370', 'n436', 'n480', 'n508', 'n515', 'n592', 'n618', 'n625']


In [46]:
df_unknown = rows_with_NaN.loc[rows_with_NaN['NID'].isin(missing_unknown)]
df_unknown.shape

(20, 34)

In [25]:
df_unknown.to_csv(path + 'unknown_locations.csv')

In [52]:
# Check with discharge dataframe
print(len(rows_with_NaN))
df_allnan = df_discharge.loc[df_discharge['NID'].isin(location_na)]
df_allnan

33


,ID,NID,LATITUDE,LONGITUDE,TYPE,HUC6,HAND,DEM,CATCHID,25-01,...,25-16,25-17,25-18,25-19,25-20,25-21,25-22,25-23,25-24,25-25
22,39377058,h23,29.549249,-95.086764,HOSPITAL,120402,4.084077,3.960619,1563608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,76177598,h34,29.544771,-95.125864,HOSPITAL,120402,7.303733,7.180261,1563618,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,32677339,h40,30.044498,-95.229984,HOSPITAL,120401,10.604908,23.573658,1468440,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,55677598,h42,29.547590,-95.117547,HOSPITAL,120402,6.168118,6.044660,1563618,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,30277338,h82,30.004777,-95.252478,HOSPITAL,120401,6.936643,20.000593,1468442,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,77477598,h93,29.543277,-95.120049,HOSPITAL,120402,6.487217,6.319258,1563618,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,6277514,h142,29.752557,-94.687447,HOSPITAL,120402,5.045572,5.045572,1524235,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143,22277550,h144,29.311201,-94.777159,HOSPITAL,120402,2.103549,2.103548,1569042,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145,22177550,h146,29.309495,-94.776991,HOSPITAL,120402,2.400525,2.400525,1569042,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
302,481947,n127,30.044685,-95.237770,NH,120401,10.757075,23.725824,1468440,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Subtract HAND

In [ ]:
df_stage.shape

In [ ]:
array_HAND = df_stage.HAND
array_discharge = df_stage[scenario_col[0]]

## Check error type 1: all discharge value in rating curve is 0

In [ ]:
## Iterate over locations
path = '/home/jupyter/tacc-work/nwm_output/'
file = 'location_maxdischarge.csv'
df_discharge = pd.read_csv(path + file, index_col = 0)
format_col = df_discharge.columns[:9].tolist()
scenario_col = df_discharge.columns[9:].tolist()
df_stage = df_discharge[format_col]

col = '25-01'
this_huc6 = 120402
col_counter = col_counter + 1
print('Running column: ', col, '(' , col_counter, '/', len(scenario_col), ')')
this_scenario = col
format_col = df_discharge.columns[:9].tolist()
format_col.append(this_scenario)
this_df_discharge = df_discharge[format_col]

In [ ]:
dict_stage = {} # NID: stage
num_locations = 0
this_df_discharge.head()

In [ ]:
this_huc6 = 120402
print('   huc6: ', this_huc6)
url = '/home/jupyter/tacc-work/hand/version-0.2.1/files/'
set_url = url + str(this_huc6) + '/'
table_name = ('hydrogeo-fulltable-%s.csv' % this_huc6 )
df_ratingcurve = pd.read_csv(set_url + table_name)
# this_df = this_df_discharge[this_df_discharge.HUC6 == this_huc6]
this_catch = 1524235
dff = df_ratingcurve[df_ratingcurve.CatchId == this_catch]

In [ ]:
print(dff.head())

In [ ]:
list_catch = df_ratingcurve.CatchId.unique()
len(list_catch)

In [ ]:
empty = []
for this_catch in list_catch:
    dff = df_ratingcurve[df_ratingcurve.CatchId == this_catch]
    
    if sum(dff['Discharge (m3s-1)']) ==0:
        empty.append(this_catch)

print(len(empty))

## Check error type 2

In [ ]:
## Iterate over locations
path = '/home/jupyter/tacc-work/nwm_output/'
file = 'location_maxdischarge.csv'
df_discharge = pd.read_csv(path + file, index_col = 0)
format_col = df_discharge.columns[:9].tolist()
scenario_col = df_discharge.columns[9:].tolist()
df_stage = df_discharge[format_col]

col = '25-02'
col_counter = col_counter + 1
print('Running column: ', col, '(' , col_counter, '/', len(scenario_col), ')')
this_scenario = col
format_col = df_discharge.columns[:9].tolist()
format_col.append(this_scenario)
this_df_discharge = df_discharge[format_col]

In [ ]:
dict_stage = {} # NID: stage
num_locations = 0
this_df_discharge.shape

In [ ]:
this_huc6 = 120701
print('   huc6: ', this_huc6)
url = '/home/jupyter/tacc-work/hand/version-0.2.1/files/'
set_url = url + str(this_huc6) + '/'
table_name = ('hydrogeo-fulltable-%s.csv' % this_huc6 )
df_ratingcurve = pd.read_csv(set_url + table_name)
df_ratingcurve.shape

In [ ]:
this_df = this_df_discharge[this_df_discharge.HUC6 == this_huc6]
this_df.head()
this_df.shape

In [ ]:
dict_stage = {} # NID: stage
for i in range(len(this_df)):
    this_location = this_df.iloc[i].NID
    this_catchid = this_df.iloc[i].CATCHID
    this_discharge = this_df.iloc[i][col]
    dff = df_ratingcurve[df_ratingcurve.CatchId == this_catchid]
    if sum(dff['Discharge (m3s-1)']) ==0:
        error_catch.append(this_catch)
        print(this_catch)
        continue

    dict_stage[this_location] = get_stage(dff, this_discharge)

print(col, num_locations)

## Check error type 3: error in get stage function when the discharge is out of range

In [ ]:
## Iterate over locations
path = '/home/jupyter/tacc-work/nwm_output/'
file = 'location_maxdischarge.csv'
df_discharge = pd.read_csv(path + file, index_col = 0)
format_col = df_discharge.columns[:9].tolist()
scenario_col = df_discharge.columns[9:].tolist()
df_stage = df_discharge[format_col]
col = '25-21'
this_Huc6 = 120701

## Check error type 4: nan

In [ ]:
## Iterate over locations
path = '/home/jupyter/tacc-work/nwm_output/'
file = 'location_maxdischarge.csv'
df_discharge = pd.read_csv(path + file, index_col = 0)
format_col = df_discharge.columns[:9].tolist()
scenario_col = df_discharge.columns[9:].tolist()
df_stage = df_discharge[format_col]

col = '25-16'
this_Huc6 = 120701
col_counter = col_counter + 1
print('Running column: ', col, '(' , col_counter, '/', len(scenario_col), ')')
this_scenario = col
format_col = df_discharge.columns[:9].tolist()
format_col.append(this_scenario)
this_df_discharge = df_discharge[format_col]

In [ ]:
dict_stage = {} # NID: stage
num_locations = 0
this_df_discharge.shape

In [ ]:
this_huc6 = 120701
print('   huc6: ', this_huc6)
url = '/home/jupyter/tacc-work/hand/version-0.2.1/files/'
set_url = url + str(this_huc6) + '/'
table_name = ('hydrogeo-fulltable-%s.csv' % this_huc6 )
df_ratingcurve = pd.read_csv(set_url + table_name)
df_ratingcurve.shape